# Polarization method that takes the sentiments of comments into account

In [ ]:
from reqs.measures import *

In [ ]:
# networks to run and print results from
network_list = [
    ['gaming', '../data/date_folders/4april/graphs/'],
    #['tarkov_network', '../data/date_folders/19march/graphs/'],
    #['CallOfDuty', '../data/date_folders/april_18/graphs/'],
    #['FIFA', '../data/date_folders/april_18/graphs/'],
    ['politics_nan', '../data/date_folders/23march_chur/graphs/'],
    ['worldnews', '../data/date_folders/23april/graphs/'],
    ['PoliticalDiscussion', '../data/date_folders/23april/graphs/'],
    ['ukpolitics', '../data/date_folders/23april/graphs/'],
    ['democrats', '../data/date_folders/23april/graphs/'],
    ['Republican', '../data/date_folders/23april/graphs/']
]

In [ ]:
for subreddit, folderpath in network_list:
    print(subreddit)
    print(driver(subreddit, folderpath))